### Creating new environement

In [ ]:
'''
conda update -n base conda &&
conda install -n base conda-libmamba-solver &&
conda config --set solver libmamba &&
source ~/anaconda3/bin/activate
'''

In [ ]:

'''
conda create -n nnunet_topcow24 python==3.9.13 anaconda -y
conda activate nnunet_topcow24
conda update -n nnunet_topcow24 conda -y

### MODIFIED THIS FILE : /home/hasna/miniconda3/envs/nnunet_topcow24/lib/python3.9/site-packages/acvl_utils/cropping_and_padding/bounding_boxes.py
'''

In [ ]:
'''
# pip install -r requirements.txt
pip install -r /home/hasna/nnUNet_dir/requirements.txt
'''

In [ ]:
'''
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia
pip install threadpoolctl==3.1.0
'''

In [ ]:
'''
export PYTHONPATH="${PYTHONPATH}:/home/hasna/miccai24_challenges/epvs_challenge/nnunet_dir"
'''

### *Preparing nnUNet*

#### *Installation*

In [ ]:
'''
TODO: Following instructions from:
https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/installation_instructions.md
pip install -e . 
'''

#### *Dataset format*

In [2]:
#libraries
import os
import sys
import json
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt

from glob import glob 
from tqdm import tqdm
from natsort import natsorted
from collections import OrderedDict
from scipy.ndimage import find_objects


##### Task 1 - CoW Segmentation

In [4]:
''' Binary MRA segmentation using MRA only '''

dataset_name = 'Dataset802_TopCoWSegBinMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_mr_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data = 1*(data>0)
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [02:52<00:00,  1.45it/s]


In [2]:
''' Binary CTA segmentation using CTA only '''

dataset_name = 'Dataset809_TopCoWSegBinCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_ct_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data = 1*(data>0)
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [01:29<00:00,  2.78it/s]


In [2]:
''' CTA and MRA for multiclass seg in the final version for CTA '''

dataset_name = 'Dataset806_TopCoWSegCTAMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.endswith('.nii.gz'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data[data==15] = 13
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [05:33<00:00,  1.33s/it]


In [ ]:
''' Multiclass MRA only '''

dataset_name = 'Dataset808_TopCoWSegMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')
### ----------------------------------------------------------------------------------------

os.makedirs(train_image_dir, exist_ok = True)
os.makedirs(train_label_dir, exist_ok = True)
os.makedirs(test_dir, exist_ok = True)
### ----------------------------------------------------------------------------------------


### Renaming according to nnUNet: 
data_path = f'/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw/{dataset_name}'
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'

subfold = ['imagesTr', 'labelsTr']
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.startswith('topcow_mr_'):                
        init_name = os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz")
        new_name = os.path.join(data_path, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz")
        # # shutil.copy(init_name, new_name)

        init_name = os.path.join(data_folder, 'cow_seg_labelsTr', sub)
        new_name = os.path.join(data_path, subfold[1], sub)
        img = nib.load(init_name)
        data = img.get_fdata()
        data[data==15] = 13
        data = data.astype(np.uint8)
        new_img = nib.Nifti1Image(data, img.affine, img.header)
        # # nib.save(new_img, new_name)


100%|██████████| 250/250 [01:07<00:00,  3.69it/s] 


#### *Preparing dataset.json file*

In [ ]:
''' Task 1: CoW Segmentation '''
### Binary CTA

dataset_name = 'Dataset809_TopCoWSegBinCTA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Binary CoW segmentation task for CTA using CTA only, trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "cta",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "cow" : 1        
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

In [5]:
''' Task 1: CoW Segmentation '''
### Multiclass CTAMRA for CTA

dataset_name = 'Dataset806_TopCoWSegCTAMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Multiclass CoW segmentation task for CTA using CTA+MRA, trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "ctamra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "BA" : 1,
        "R-PCA" : 2,
        "L-PCA" : 3,
        "R-ICA" : 4,
        "R-MCA" : 5,
        "L-ICA" : 6,
        "L-MCA" : 7,
        "R-Pcom" : 8,
        "L-Pcom" : 9,
        "Acom" : 10,
        "R-ACA" : 11,
        "L-ACA" : 12,
        "3rd-A2" : 13
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


In [ ]:
''' Task 1: CoW Segmentation '''
### Binary MRA

dataset_name = 'Dataset802_TopCoWSegBinMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Binary CoW segmentation task for MRA using MRA only, trained for all folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "mra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "cow" : 1        
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

In [4]:
''' Task 1: CoW Segmentation '''
### Multiclass MRA

dataset_name = 'Dataset808_TopCoWSegMRA'
nnunet_basedir = "/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir"
dataset_folder_name = os.path.join(nnunet_basedir, 'dataset/nnUNet_raw/', dataset_name)
train_image_dir = os.path.join(dataset_folder_name,'imagesTr')
train_label_dir = os.path.join(dataset_folder_name,'labelsTr')
test_dir = os.path.join(dataset_folder_name,'imagesTs')

dataset_json_path = os.path.join(dataset_folder_name, 'dataset.json')
overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
json_file_exist = False

if os.path.exists(dataset_json_path):
    print('dataset.json already exist!')
    json_file_exist = True

if json_file_exist==False or overwrite_json_file:
    json_dict = OrderedDict()
    json_dict['dataset_name'] = dataset_name
    json_dict['description'] = "Multiclass CoW segmentation task for MRA, trained on 5 folds for the TopCoW2024 Challenge" #"Binary CoW roi Detection task using CTA and Binary segmentation mask from the multiclass GT, trained for 5 folds for the TopCoW2024 Challenge"
    json_dict['tensor_image_size'] = "3D"
    json_dict['file_ending'] = ".nii.gz"

    #you may mention more than one modality
    json_dict['channel_names'] = {
        "0": "mra",
    }
    #labels+1 should be mentioned for all the labels in the dataset
    json_dict['labels'] = {
        "background": 0,
        "BA" : 1,
        "R-PCA" : 2,
        "L-PCA" : 3,
        "R-ICA" : 4,
        "R-MCA" : 5,
        "L-ICA" : 6,
        "L-MCA" : 7,
        "R-Pcom" : 8,
        "L-Pcom" : 9,
        "Acom" : 10,
        "R-ACA" : 11,
        "L-ACA" : 12,
        "3rd-A2" : 13
    }

    train_ids = sorted(os.listdir(train_label_dir))
    test_ids = sorted(os.listdir(test_dir))
    json_dict['numTraining'] = len(train_ids)
    json_dict['numTesting'] = len(test_ids)

    # no modality in train image and labels in dataset.json 
    json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

    # removing the modality from test image name to be saved in dataset.json
    json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids]

    with open(dataset_json_path, 'w') as f:
        json.dump(json_dict, f, indent=4, sort_keys=True)

    if os.path.exists(dataset_json_path):
        if json_file_exist==False:
            print('dataset.json created!')
        else: 
            print('dataset.json overwritten!')

dataset.json created!


#### Extra information

In [9]:
''' Information about Spacings and Sizes of all modalities '''

dataset_name = 'Dataset701_TopCoWDetCTA' 
# Renaming according to nnUNet: 
data_folder = '/home/hasna/datasets/TopCoW2024_Data_Release'
subfold = ['imagesTr', 'cow_seg_labelsTr']

mri_spacing, seg_spacing = [], []
mri_size, seg_size = [], []
labels = []
fnames = []

# imageTr:
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'cow_seg_labelsTr')))):
    if sub.endswith('.nii.gz'):        
        mri = sitk.ReadImage(os.path.join(data_folder, subfold[0], f"{sub.split('.')[0]}_0000.nii.gz"))
        mri_spacing.append(mri.GetSpacing())
        mri_size.append(mri.GetSize())
        
        seg = sitk.ReadImage(os.path.join(data_folder, subfold[1], sub))
        seg_spacing.append(seg.GetSpacing())
        seg_size.append(seg.GetSize())

        fnames.append(sub.split('.')[0])
        labels.append(sitk.GetArrayFromImage(seg).max())

df = pd.DataFrame({'filename':fnames, 'mri_spacing':mri_spacing, 'seg_spacing':seg_spacing, 'mri_size':mri_size, 'seg_size':seg_size, 'max_label':labels})
# df.to_csv('/home/hasna/miccai24_challenges/topcow_challenge/topcow24_dataset_spacing_size.tsv', sep='\t')

100%|██████████| 250/250 [03:22<00:00,  1.23it/s]


In [3]:
''' Information about Spacings and Sizes of all modalities '''

data_folder = '/home/hasna/datasets/crown_20_mr_01062023'

mri_spacing, seg_spacing = [], []
mri_size, seg_size = [], []
labels = []
fnames = []

# imageTr:
for sub in tqdm(natsorted(os.listdir(os.path.join(data_folder, 'mul_labelsTr')))):
    if sub.startswith('crown_mr_whole'):        
        mri = sitk.ReadImage(os.path.join(data_folder, 'imagesTr', f"{sub.split('.')[0]}_0000.nii.gz"))
        mri_spacing.append(mri.GetSpacing())
        mri_size.append(mri.GetSize())
        
        seg = sitk.ReadImage(os.path.join(data_folder, 'mul_labelsTr', sub))
        seg_spacing.append(seg.GetSpacing())
        seg_size.append(seg.GetSize())

        fnames.append(sub.split('.')[0])
        labels.append(sitk.GetArrayFromImage(seg).max())

df = pd.DataFrame({'filename':fnames, 'mri_spacing':mri_spacing, 'seg_spacing':seg_spacing, 'mri_size':mri_size, 'seg_size':seg_size, 'max_label':labels})
# df.to_csv('/home/hasna/miccai24_challenges/topcow_challenge/crown23_dataset_spacing_size.tsv', sep='\t')

100%|██████████| 40/40 [00:18<00:00,  2.22it/s]


### *Training nnUNet*

In [ ]:
'''
export nnUNet_raw='/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/nnUNet_raw'
export nnUNet_preprocessed='/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/dataset/preprocessed'
export nnUNet_results='/home/hasna/miccai24_challenges/topcow_challenge/nnunet_dir/datasetnnUNet_trained_models'


>>> For task 1:
nnUNetv2_extract_fingerprint -d 802 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 802 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 802 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...

nnUNetv2_extract_fingerprint -d 806 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 806 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 806 -c 3d_fullres_ps -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...

nnUNetv2_extract_fingerprint -d 808 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 808 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 808 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...

nnUNetv2_extract_fingerprint -d 809 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'
nnUNetv2_plan_experiment -d 809 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'
nnUNetv2_preprocess -d 809 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...

'''

"\nexport nnUNet_raw='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/nnUNet_raw'\nexport nnUNet_preprocessed='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/dataset/preprocessed'\nexport nnUNet_results='/home/hasna/miccai24_challenges/topcow_challenge_final/nnunet_dir/datasetnnUNet_trained_models'\n\n\n>>> For task 1:\nnnUNetv2_extract_fingerprint -d 802 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'\nnnUNetv2_plan_experiment -d 802 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'\nnnUNetv2_preprocess -d 802 -c 3d_fullres -np 8 --verify_dataset_integrity   # --> to get the 'gt_segmentations' and 'nnUNetPlans_3d_fullres' ...\n\n\nnnUNetv2_extract_fingerprint -d 806 --verify_dataset_integrity   # --> to get the 'dataset_fingerprint.json'\nnnUNetv2_plan_experiment -d 806 --verify_dataset_integrity   # --> to get the 'dataset.json' and 'nnUNetPlans.json'\nnnUNetv2_preprocess -d 806 -c

In [ ]:
''' For task 1: -------------------------------------------------------------------------------------------
>>> Training for MRA:
>> [Binary - MRA only]:
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 802 3d_fullres all -tr nnUNetTrainerSkeletonRecall

>> [Multiclass - MRA only]:
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 808 3d_fullres 0 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_0/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 808 3d_fullres 1 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_1/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 808 3d_fullres 2 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_2/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 808 3d_fullres 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_3/checkpoint_best.pth
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 808 3d_fullres 4 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring -pretrained_weights /home/hasna/miccai24_challenges/topcow_challenge/Dataset101_Mulseg/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_4/checkpoint_best.pth


>>> Training for CTA:
>> [Binary - CTA only]:
CUDA_VISIBLE_DEVICES=2 nnUNetv2_train 809 3d_fullres all -tr nnUNetTrainerSkeletonRecall --val_best

>> [Multiclass - CTA+MRA]
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 806 3d_fullres_ps 0 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 806 3d_fullres_ps 1 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring
CUDA_VISIBLE_DEVICES=3 nnUNetv2_train 806 3d_fullres_ps 2 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring
CUDA_VISIBLE_DEVICES=0 nnUNetv2_train 806 3d_fullres_ps 3 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring
CUDA_VISIBLE_DEVICES=1 nnUNetv2_train 806 3d_fullres_ps 4 -tr nnUNetTrainerSkeletonRecallBinDiceNoMirroring

'''

### Docker

In [ ]:
'''
cd /home/hasna/miccai24_challenges/TopCoW_Algo_Submission/task-1-seg
bash test_run.sh
bash save.sh
'''